In [515]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

DATA COLLECTION

Datos de Yahoo Finance

In [516]:
sp500 = yf.Ticker("^GSPC")
gold = yf.Ticker("^GC=F")
oil= yf.Ticker("CL=F")
brent= yf.Ticker("BZ=F")

Preparación y limpieza del Sp500

Para el Sp500 utilizo los datos a partir del año 2000, ya que en años anteriores la composición por industria del SP500 era bastante diferente. A partir de ese año hay importnate participación del sector teconología.

In [517]:
sp500 = sp500.history(period="max")
sp500 =sp500.loc['2000-01-01':].copy ()
oil = oil.history(period="max")
oil = oil.loc['2000-01-01':].copy ()
brent = brent.history(period="max") 
brent = brent.loc['2000-01-01':].copy ()

In [518]:
sp500.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-03 00:00:00-05:00,1469.250000,1478.000000,1438.359985,1455.219971,931800000,0.0,0.0
2000-01-04 00:00:00-05:00,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,0.0,0.0
2000-01-05 00:00:00-05:00,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,0.0,0.0
2000-01-06 00:00:00-05:00,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,0.0,0.0
2000-01-07 00:00:00-05:00,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,0.0,0.0


In [519]:
sp500 = sp500.drop(columns=['Dividends', 'Stock Splits'])
sp500.index= sp500.index.tz_localize(None) # remove timezone

In [520]:
#Calculo la media 
# Periodos para las medias móviles
moving_avg_periods = [5,10, 15]

# Calcular y asignar cada media móvil utilizando un bucle
for period in moving_avg_periods:
    sp500[f'{period}d_moving_avg'] = sp500['Close'].rolling(window=period).mean()

# Mostrar las primeras filas para verificar las nuevas columnas
sp500.head()



,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,15d_moving_avg
Date,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN


In [521]:
#Calculo el promedio móvil ponderado de los ultimas 2 semanas

# Definimos los pesos decrecientes para los últimos 10 días hábiles.
weights = np.arange(1, 11)[::-1]  # Esto crea un array de numpy: [10, 9, ..., 2, 1]

# Calculamos el WMA para cada ventana de 10 días hábiles.
# Usamos apply para aplicar una función personalizada que calcula el WMA de manera vectorizada.
sp500['10d_wma'] = sp500['Close'].rolling(window=10).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)

# Ahora, el DataFrame 'sp500' contiene una nueva columna '10d_wma' con el WMA de 10 días.
# Muestra los primeros registros para verificar.
print(sp500[['Close', '10d_wma']].head(20))


                  Close      10d_wma
Date                                
2000-01-03  1455.219971          NaN
2000-01-04  1399.420044          NaN
2000-01-05  1402.109985          NaN
2000-01-06  1403.449951          NaN
2000-01-07  1441.469971          NaN
2000-01-10  1457.599976          NaN
2000-01-11  1438.560059          NaN
2000-01-12  1432.250000          NaN
2000-01-13  1449.680054          NaN
2000-01-14  1465.150024  1428.005995
2000-01-18  1455.140015  1424.235638
2000-01-19  1455.900024  1431.637631
2000-01-20  1445.569946  1439.340725
2000-01-21  1441.359985  1447.489462
2000-01-24  1401.530029  1447.999288
2000-01-25  1410.030029  1444.711477
2000-01-26  1404.089966  1444.258740
2000-01-27  1398.560059  1444.340743
2000-01-28  1360.160034  1439.626008
2000-01-31  1394.459961  1430.813277


In [522]:
# Calcular el momentum como el cambio día a día en la columna 'Close'
sp500['Momentum'] = sp500['Close'].diff()

print(sp500.head())  # Mostrar las primeras filas para verificar la nueva columna

                   Open         High          Low        Close      Volume  \
Date                                                                         
2000-01-03  1469.250000  1478.000000  1438.359985  1455.219971   931800000   
2000-01-04  1455.219971  1455.219971  1397.430054  1399.420044  1009000000   
2000-01-05  1399.420044  1413.270020  1377.680054  1402.109985  1085500000   
2000-01-06  1402.109985  1411.900024  1392.099976  1403.449951  1092300000   
2000-01-07  1403.449951  1441.469971  1400.729980  1441.469971  1225200000   

            5d_moving_avg  10d_moving_avg  15d_moving_avg  10d_wma   Momentum  
Date                                                                           
2000-01-03            NaN             NaN             NaN      NaN        NaN  
2000-01-04            NaN             NaN             NaN      NaN -55.799927  
2000-01-05            NaN             NaN             NaN      NaN   2.689941  
2000-01-06            NaN             NaN            

El Stochastic %K es una medida de la posición relativa del precio de cierre actual en relación con el rango alto-bajo durante un número específico de períodos pasados. La fórmula para calcular el Stochastic %K es la siguiente:

% K = [(C-LLn)/(HHn-LLn)]x 100� 

donde:


C es el precio de cierre más reciente,

LLn es el precio más bajo de los prcios bajos (Low) en los últimos n períodos,
Hn  es el precio más alto de lso pecios altos (High) en los últimos n períodos,
n es el número de períodos utilizados para el cálculo.
El valor de n varía según el análisis, pero los valores comunes incluyen 14, 20 o cualquier otro valor dependiendo de la estrategia de trading

In [523]:
# Calcular el Stochastic %K para cada fila basado en los 20 días laborables previos
# Crear una función para encapsular el cálculo
#def calculate_stochastic_k(row_index, df):
    #if row_index < 19:  # Asegurar que haya suficientes datos para los primeros 19 registros
        #return None
    #else:
        #period = df['Close'][row_index-19:row_index+1]  # Seleccionar los últimos 20 registros hasta el índice actual
        #low_20 = period.min()
        #high_20 = period.max()
        #close_recent = df['Close'].iloc[row_index]
        #stochastic_k = ((close_recent - low_20) / (high_20 - low_20)) * 100
        #return stochastic_k

# Aplicar la función a cada fila
# Nota: Esto puede ser lento para DataFrames muy grandes debido al enfoque iterativo
#sp500['Stochastic_%K'] = [calculate_stochastic_k(i, sp500) for i in range(len(sp500))]

#print(sp500.tail())  # Mostrar las últimas filas para verificar la nueva columna
#sp500.head


# Crear una función para encapsular el cálculo del Stochastic %K
def calculate_stochastic_k(row_index, df):
    if row_index < 19:  # Asegurar que haya suficientes datos para los primeros 19 registros
        return None
    else:
        # Seleccionar los últimos 20 registros hasta el índice actual para las columnas 'High' y 'Low'
        high_period = df['High'][row_index-19:row_index+1]
        low_period = df['Low'][row_index-19:row_index+1]
        low_20 = low_period.min()
        high_20 = high_period.max()
        close_recent = df['Close'].iloc[row_index]
        
        # Calcular el Stochastic %K
        stochastic_k = ((close_recent - low_20) / (high_20 - low_20)) * 100
        return stochastic_k

# Aplicar la función a cada fila
sp500['Stochastic_%K'] = [calculate_stochastic_k(i, sp500) for i in range(len(sp500))]

# Mostrar las últimas filas para verificar la nueva columna
print(sp500.tail())


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-09  5217.029785  5224.810059  5160.779785  5209.910156  3400680000   
2024-04-10  5167.879883  5178.430176  5138.700195  5160.640137  3845930000   
2024-04-11  5172.950195  5211.779785  5138.770020  5199.060059  3509380000   
2024-04-12  5171.509766  5175.029785  5107.939941  5123.410156  3963220000   
2024-04-15  5149.669922  5168.430176  5061.049805  5063.640137  1375658000   

            5d_moving_avg  10d_moving_avg  15d_moving_avg      10d_wma  \
Date                                                                     
2024-04-09    5195.068066     5213.134082     5215.224707  5220.404821   
2024-04-10    5184.898047     5208.840088     5214.033398  5221.361186   
2024-04-11    5195.268066     5203.897070     5212.329395  5213.253338   
2024-04-12    5179.082129     5190.803076     5204.454753  5201.699334   
2024-04-1

El indicador Stochastic %D es una media móvil del indicador Stochastic %K. Usualmente se calcula como la media móvil simple de los últimos 3 valores de %K. Para calcular el %D para las últimas 20 observaciones de tu DataFrame sp500, primero necesitaríamos calcular el %K como se discutió anteriormente, y luego aplicar una media móvil sobre esos valores de %K.

In [524]:

# Calcular el Stochastic %D como la media móvil simple de 3 días de %K
sp500['Stochastic_%D'] = sp500['Stochastic_%K'].rolling(window=3).mean()

# Si solo necesitas las últimas 20 observaciones para el %D, puedes hacer lo siguiente:
last_20_stochastic_D = sp500['Stochastic_%D'][-20:]

print(last_20_stochastic_D)
sp500.head(9)

Date
2024-03-18    79.337650
2024-03-19    83.178508
2024-03-20    92.788500
2024-03-21    95.054287
2024-03-22    92.134690
2024-03-25    85.412135
2024-03-26    79.219710
2024-03-27    81.554747
2024-03-28    86.874143
2024-04-01    92.882218
2024-04-02    85.479659
2024-04-03    76.922842
2024-04-04    57.726515
2024-04-05    55.575326
2024-04-08    53.829101
2024-04-09    64.992990
2024-04-10    54.961573
2024-04-11    53.283501
2024-04-12    35.318808
2024-04-15    24.051921
Name: Stochastic_%D, dtype: float64


,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,15d_moving_avg,10d_wma,Momentum,Stochastic_%K,Stochastic_%D
Date,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN,-55.799927,NaN,NaN
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN,2.689941,NaN,NaN
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN,1.339966,NaN,NaN
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN,38.020020,NaN,NaN
2000-01-10,1441.469971,1464.359985,1441.469971,1457.599976,1064800000,1420.809985,NaN,NaN,NaN,16.130005,NaN,NaN
2000-01-11,1457.599976,1458.660034,1434.420044,1438.560059,1014000000,1428.637988,NaN,NaN,NaN,-19.039917,NaN,NaN
2000-01-12,1438.560059,1442.599976,1427.079956,1432.250000,974600000,1434.665991,NaN,NaN,NaN,-6.310059,NaN,NaN
2000-01-13,1432.250000,1454.199951,1432.250000,1449.680054,1030400000,1443.912012,NaN,NaN,NaN,17.430054,NaN,NaN



El Índice de Fuerza Relativa (RSI, por sus siglas en inglés) es un indicador de momento que mide la magnitud de los movimientos recientes de precios para evaluar condiciones de sobrecompra o sobreventa en el precio de un activo. Se calcula utilizando la fórmula:

RSI=100− 100/(1+RS)
 

donde 
�
�
RS es la relación media de días de cierre hacia arriba entre la media de días de cierre hacia abajo. Para calcularlo para los últimos 20 y 10 días, necesitamos seguir los siguientes pasos para cada ventana de tiempo:

Calcular los cambios diarios en el precio de cierre.
Separar estos cambios en ganancias (aumentos) y pérdidas (disminuciones).
Calcular el promedio de ganancias y el promedio de pérdidas sobre el período de interés (20 días y 10 días, respectivamente).
Calcular el RS, que es el promedio de ganancias dividido por el promedio de pérdidas.
Utilizar la fórmula del RSI para obtener el valor final.

In [525]:
# Separar las ganancias (positivas) de las pérdidas (negativas)
gain = sp500['Momentum'].mask(sp500['Momentum'] > 0, 0)
loss = -sp500['Momentum'].mask(sp500['Momentum'] < 0, 0)

# Calcular el RSI para los últimos 20 y 10 días
window_lengths = [20, 10]

for length in window_lengths:
    # Calcular el promedio de ganancias y pérdidas
    avg_gain = gain.rolling(window=length, min_periods=1).mean()
    avg_loss = loss.rolling(window=length, min_periods=1).mean()
    
    # Calcular RS
    rs = avg_gain / avg_loss
    
    # Calcular RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Adjuntar al DataFrame
    sp500[f'RSI_{length}'] = rsi

# Mostrar las últimas 20 filas para verificar las nuevas columnas
print(sp500.tail(20))



                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-03-18  5154.770020  5175.600098  5145.470215  5149.419922  4036220000   
2024-03-19  5139.089844  5180.310059  5131.589844  5178.509766  4031760000   
2024-03-20  5181.689941  5226.189941  5171.549805  5224.620117  4064850000   
2024-03-21  5253.430176  5261.100098  5240.660156  5241.529785  4207730000   
2024-03-22  5242.479980  5246.089844  5229.870117  5234.180176  3374700000   
2024-03-25  5219.520020  5229.089844  5216.089844  5218.189941  3331360000   
2024-03-26  5228.850098  5235.160156  5203.419922  5203.580078  3871790000   
2024-03-27  5226.310059  5249.259766  5213.919922  5248.490234  3850500000   
2024-03-28  5248.029785  5264.850098  5245.819824  5254.350098  3998270000   
2024-04-01  5257.970215  5263.950195  5229.200195  5243.770020  3325930000   
2024-04-02  5204.290039  5208.339844  5184.049805  5205.810059  


El indicador Larry Williams %R es una medida de momentum que se utiliza para identificar niveles de sobrecompra y sobreventa en un mercado. Se calcula como un porcentaje que refleja la posición del precio de cierre relativo a su rango más alto y más bajo durante un número específico de períodos pasados. La fórmula del Larry Williams %R es:

%R= [(Hn-C)/(Hn-Ln)]x100

donde:

C es el precio de cierre actual,
Hn  es el precio más alto en los últimos n períodos,
Ln es el precio más bajo en los últimos 
�
n períodos,
�
n es el número de períodos utilizados para el cálculo.
Generalmente, se utiliza un período de 14 días para el cálculo de este indicador, pero se puede ajustar según las necesidades del análisis. Para agregar el Larry Williams %R al DataFrame sp500 para los últimos 14 días, puedes usar el siguiente código:

In [526]:
# Asumiendo que sp500 es tu DataFrame y tiene columnas 'High', 'Low', y 'Close' con los precios más altos, más bajos y de cierre

# Configurar el número de períodos para el cálculo de Williams %R
n = 10

# Calcular el precio más alto y el más bajo de los últimos n días
high_n = sp500['High'].rolling(window=n).max()
low_n = sp500['Low'].rolling(window=n).min()

# Calcular el Williams %R
sp500['Williams_%R'] = ((high_n - sp500['Close']) / (high_n - low_n)) * -100

# Mostrar las últimas filas para verificar la nueva columna
print(sp500.tail())


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-09  5217.029785  5224.810059  5160.779785  5209.910156  3400680000   
2024-04-10  5167.879883  5178.430176  5138.700195  5160.640137  3845930000   
2024-04-11  5172.950195  5211.779785  5138.770020  5199.060059  3509380000   
2024-04-12  5171.509766  5175.029785  5107.939941  5123.410156  3963220000   
2024-04-15  5149.669922  5168.430176  5061.049805  5063.640137  1375658000   

            5d_moving_avg  10d_moving_avg  15d_moving_avg      10d_wma  \
Date                                                                     
2024-04-09    5195.068066     5213.134082     5215.224707  5220.404821   
2024-04-10    5184.898047     5208.840088     5214.033398  5221.361186   
2024-04-11    5195.268066     5203.897070     5212.329395  5213.253338   
2024-04-12    5179.082129     5190.803076     5204.454753  5201.699334   
2024-04-1

In [527]:
#https://school.stockcharts.com/doku.php?id=technical_indicators:moving_average_convergence_divergence_macd


# Calcular las EMAs de 12 y 26 períodos para la columna 'Close'
ema12 = sp500['Close'].ewm(span=12, adjust=False).mean()
ema26 = sp500['Close'].ewm(span=26, adjust=False).mean()

# Calcular el MACD restando la EMA de 26 períodos de la EMA de 12 períodos
sp500['MACD'] = ema12 - ema26

# Calcular la línea de señal como la EMA de 9 períodos del MACD
sp500['Signal_Line'] = sp500['MACD'].ewm(span=9, adjust=False).mean()

# Mostrar las últimas filas para verificar las nuevas columnas
print(sp500.tail())
sp500.head()


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-09  5217.029785  5224.810059  5160.779785  5209.910156  3400680000   
2024-04-10  5167.879883  5178.430176  5138.700195  5160.640137  3845930000   
2024-04-11  5172.950195  5211.779785  5138.770020  5199.060059  3509380000   
2024-04-12  5171.509766  5175.029785  5107.939941  5123.410156  3963220000   
2024-04-15  5149.669922  5168.430176  5061.049805  5063.640137  1375658000   

            5d_moving_avg  10d_moving_avg  15d_moving_avg      10d_wma  \
Date                                                                     
2024-04-09    5195.068066     5213.134082     5215.224707  5220.404821   
2024-04-10    5184.898047     5208.840088     5214.033398  5221.361186   
2024-04-11    5195.268066     5203.897070     5212.329395  5213.253338   
2024-04-12    5179.082129     5190.803076     5204.454753  5201.699334   
2024-04-1

,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,15d_moving_avg,10d_wma,Momentum,Stochastic_%K,Stochastic_%D,RSI_20,RSI_10,Williams_%R,MACD,Signal_Line
Date,,,,,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN,-55.799927,NaN,NaN,100.000000,100.000000,NaN,-4.451276,-0.890255
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN,2.689941,NaN,NaN,95.401013,95.401013,NaN,-7.673434,-2.246891
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN,1.339966,NaN,NaN,93.264385,93.264385,NaN,-10.003579,-3.798229
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN,38.020020,NaN,NaN,57.026071,57.026071,NaN,-8.682254,-4.775034


In [528]:
# Calcular los retornos diarios
sp500['Daily_Returns'] = sp500['Close'].pct_change()

# Crear un "lag" de 1 día para los retornos diarios
sp500['Lag 1_Returns'] = sp500['Daily_Returns'].shift(+1)
sp500['Lag 2_Returns'] = sp500['Daily_Returns'].shift(+2)
sp500['Next_Day_Returns'] = sp500['Daily_Returns'].shift(-1)

# Crear la columna 'TARGET' basada en 'Lagged_Daily_Returns' - Quiero predecir si el retorno del día siguiente será positivo o negativo basado con respecto al día actual en el que tomo la decision al final del día
sp500['TARGET'] = np.where(sp500['Next_Day_Returns'] > 0.000, 1, 0)

# Mostrar las primeras filas para verificar las nuevas columnas
sp500.head()



,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,15d_moving_avg,10d_wma,Momentum,...,RSI_20,RSI_10,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns,Next_Day_Returns,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,-0.038345,0
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN,-55.799927,...,100.000000,100.000000,NaN,-4.451276,-0.890255,-0.038345,NaN,NaN,0.001922,1
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN,2.689941,...,95.401013,95.401013,NaN,-7.673434,-2.246891,0.001922,-0.038345,NaN,0.000956,1
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN,1.339966,...,93.264385,93.264385,NaN,-10.003579,-3.798229,0.000956,0.001922,-0.038345,0.027090,1
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN,38.020020,...,57.026071,57.026071,NaN,-8.682254,-4.775034,0.027090,0.000956,0.001922,0.011190,1


In [529]:
# Contar los valores nulos en cada columna
null_counts = sp500.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts)

# Eliminar las filas con valores nulos o NaN
sp500_clean = sp500.dropna()
# Contar los valores nulos en cada columna
null_counts_clean = sp500_clean.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_clean)

sp500_clean.to_csv("sp500_clean.csv")

Conteo de valores nulos por columna antes de limpiar:
 Open                 0
High                 0
Low                  0
Close                0
Volume               0
5d_moving_avg        4
10d_moving_avg       9
15d_moving_avg      14
10d_wma              9
Momentum             1
Stochastic_%K       19
Stochastic_%D       21
RSI_20               1
RSI_10               1
Williams_%R          9
MACD                 0
Signal_Line          0
Daily_Returns        1
Lag 1_Returns        2
Lag 2_Returns        3
Next_Day_Returns     1
TARGET               0
dtype: int64
Conteo de valores nulos por columna antes de limpiar:
 Open                0
High                0
Low                 0
Close               0
Volume              0
5d_moving_avg       0
10d_moving_avg      0
15d_moving_avg      0
10d_wma             0
Momentum            0
Stochastic_%K       0
Stochastic_%D       0
RSI_20              0
RSI_10              0
Williams_%R         0
MACD                0
Signal_Line        

In [530]:
import requests


# FRED API endpoint
url = "https://api.stlouisfed.org/fred/series/observations"

# List of series IDs to be downloaded
# CREDIT RISK AND RISK APPETITE VARIABLES
# DAAA = Moody's Seasoned Aaa Corporate Bond Yield
# AAA10Y = Moody's Seasoned Aaa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
# DBAA = Moody's Seasoned Baa Corporate Bond Yield
# BAA10Y = Moody's Seasoned Baa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
# BAMLH0A0HYM2 = ICE BofA US High Yield Index Option-Adjusted Spread 
# BAMLH0A2HYB = ICE BofA Single-B US High Yield Index Option-Adjusted Spread 
# BAMLH0A3HYC = ICE BofA CCC & Lower US High Yield Index Option-Adjusted Spread
# BAMLHE00EHYIOAS = ICE BofA Euro High Yield Index Option-Adjusted Spread -  

# INTEREST RATE VARIABLES
# T10Y2Y = 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity
# T10Y3M = 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity  
# T5YIE = 5-Year Breakeven Inflation Rate   

# EXCHANGE RATES
# DEXUSEU = U.S. Dollars to Euro Spot Exchange Rate - value of one U.S. Dollar in Euros
# DEXCHUS = Chinese Yuan Renminbi to U.S. Dollar Spot Exchange Rate - value of one Chinese Yuan Renminbi in U.S. Dollars
# DEXCAUS = Canadian Dollars to U.S. Dollar Spot Exchange Rate - value of one Canadian Dollar in U.S. Dollars
# DEXMXUS= Mexican Pesos to U.S. Dollar Spot Exchange Rate - value of one Mexican Peso in U.S. Dollars

# VOLATILITY INDECES
# VIXCLS = CBOE Volatility Index - VIX measures market expectation of near term volatility conveyed by stock index option prices.
#(GVZCLS)= CBOE Gold ETF Volatility Index 
series_ids = ['DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS','GVZCLS']

# Your FRED API key
api_key = '7de04d83472c6affd4e0a1505211d3b9'

# Initialize an empty dictionary to store DataFrames
dfs = {}

# Loop through each series ID
for series_id in series_ids:
    # API parameters
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
    }
    
    # Make the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        data = response.json()
        # Extract data for DataFrame
        dates = [item['date'] for item in data['observations']]
        values = [item['value'] for item in data['observations']]
        # Create DataFrame and convert types
        temp_df = pd.DataFrame({'Date': dates, series_id: values})
        temp_df['Date'] = pd.to_datetime(temp_df['Date'])
        temp_df[series_id] = pd.to_numeric(temp_df[series_id], errors='coerce')
        # Add the DataFrame to the dictionary
        dfs[series_id] = temp_df.set_index('Date')
    else:
        print(f"Failed to retrieve data for {series_id}: {response.status_code}")

# Combine all the series DataFrames into one DataFrame
macroeconomic_df = pd.concat(dfs.values(), axis=1)




In [531]:
#Sacar los cambios diarios de las variables macroeconomicas

changes_daily = macroeconomic_df.diff()

# Renombrar las columnas para indicar que representan cambios diarios
changes_daily.columns = [f"{col}_daily_change" for col in macroeconomic_df.columns]

# Concatenar los cambios diarios de vuelta al DataFrame original
macroeconomic_df = pd.concat([macroeconomic_df, changes_daily], axis=1)


In [532]:
# Contar los valores nulos en cada columna
null_counts_2 =macroeconomic_df.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts)

# Eliminar las filas con valores nulos o NaN
macroeconomic_df_clean = macroeconomic_df.dropna()
# Contar los valores nulos en cada columna
null_counts_clean = macroeconomic_df_clean.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_clean)

macroeconomic_df_clean.to_csv("macroeconomic_df_clean.csv")

Conteo de valores nulos por columna antes de limpiar:
 Open                 0
High                 0
Low                  0
Close                0
Volume               0
5d_moving_avg        4
10d_moving_avg       9
15d_moving_avg      14
10d_wma              9
Momentum             1
Stochastic_%K       19
Stochastic_%D       21
RSI_20               1
RSI_10               1
Williams_%R          9
MACD                 0
Signal_Line          0
Daily_Returns        1
Lag 1_Returns        2
Lag 2_Returns        3
Next_Day_Returns     1
TARGET               0
dtype: int64
Conteo de valores nulos por columna antes de limpiar:
 DAAA                            0
AAA10Y                          0
DBAA                            0
BAA10Y                          0
BAMLH0A0HYM2                    0
BAMLH0A2HYB                     0
BAMLH0A3HYC                     0
BAMLHE00EHYIOAS                 0
T10Y2Y                          0
T10Y3M                          0
T5YIE                          

OJO ----- FALTA AGREGAR OIL

In [533]:
# Unir macroeconomic_df_clean con sp500_clean basado en sus índices de fecha
AllVar_df = macroeconomic_df_clean.join(sp500_clean, how='inner')


# Contar los valores nulos en cada columna
null_counts_clean_AllVar = AllVar_df.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_clean)

print(AllVar_df.index)
AllVar_df = AllVar_df.sort_index()

Conteo de valores nulos por columna antes de limpiar:
 DAAA                            0
AAA10Y                          0
DBAA                            0
BAA10Y                          0
BAMLH0A0HYM2                    0
BAMLH0A2HYB                     0
BAMLH0A3HYC                     0
BAMLHE00EHYIOAS                 0
T10Y2Y                          0
T10Y3M                          0
T5YIE                           0
DEXUSEU                         0
DEXCHUS                         0
DEXCAUS                         0
DEXMXUS                         0
VIXCLS                          0
GVZCLS                          0
DAAA_daily_change               0
AAA10Y_daily_change             0
DBAA_daily_change               0
BAA10Y_daily_change             0
BAMLH0A0HYM2_daily_change       0
BAMLH0A2HYB_daily_change        0
BAMLH0A3HYC_daily_change        0
BAMLHE00EHYIOAS_daily_change    0
T10Y2Y_daily_change             0
T10Y3M_daily_change             0
T5YIE_daily_change         

LOGIT

In [534]:
#  Pasar los nombres de las columnas a excluir como una lista al argumento `columns`

fecha_inicio = '2012-01-01'
AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]

X_logit=AllVar_df_Shortened.drop(columns=['Next_Day_Returns','TARGET', 'DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS_daily_change','GVZCLS_daily_change'])

Y_logit= AllVar_df_Shortened['TARGET']

X_logit.head()

,VIXCLS,GVZCLS,DAAA_daily_change,AAA10Y_daily_change,DBAA_daily_change,BAA10Y_daily_change,BAMLH0A0HYM2_daily_change,BAMLH0A2HYB_daily_change,BAMLH0A3HYC_daily_change,BAMLHE00EHYIOAS_daily_change,...,Stochastic_%K,Stochastic_%D,RSI_20,RSI_10,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-04,22.22,22.08,0.05,0.02,0.04,0.01,-0.06,-0.08,-0.05,-0.15,...,91.100369,88.113921,45.629419,18.545107,-9.277496,11.361921,7.542044,0.000188,0.015474,-0.004291
2012-01-05,21.48,22.34,0.02,0.00,-0.01,-0.03,0.00,0.00,0.04,0.01,...,95.671810,92.526922,45.166664,25.809245,-6.459694,12.478079,8.529251,0.002944,0.000188,0.015474
2012-01-06,20.63,21.30,-0.06,-0.02,-0.04,0.00,0.00,-0.01,-0.02,-0.07,...,91.720442,92.830874,46.416378,29.466339,-16.650202,12.951102,9.413621,-0.002537,0.002944,0.000188
2012-01-09,21.07,21.47,0.01,0.01,0.01,0.01,-0.01,-0.01,-0.03,0.02,...,95.233989,94.208747,38.997313,32.346000,-10.895070,13.404647,10.211827,0.002262,-0.002537,0.002944
2012-01-10,20.69,21.51,0.00,-0.02,-0.01,-0.03,-0.07,-0.06,-0.15,-0.07,...,95.344876,94.099769,40.831147,32.324606,-9.159368,14.515035,11.072468,0.008886,0.002262,-0.002537


In [535]:
#importo las libreris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [536]:

# Asumiendo que ya has definido X_logit (variables independientes) y Y_logit (variable dependiente), y están sin la columna 'Fecha'
X_train, X_test, Y_train, Y_test = train_test_split(X_logit, Y_logit, test_size=0.2, shuffle=False)  # shuffle=False para mantener el orden


scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Crear una instancia del modelo de Regresión Logística
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo con el conjunto de entrenamiento
model.fit(X_train_scaled, Y_train)



LogisticRegression(max_iter=1000)

In [537]:
# Realizar predicciones en el conjunto de prueba
Y_pred = model.predict(X_test_scaled )

In [538]:
# Imprimir reporte de clasificación
print(classification_report(Y_test, Y_pred))

# Imprimir matriz de confusión
print(confusion_matrix(Y_test, Y_pred))


              precision    recall  f1-score   support

           0       0.52      0.19      0.28       281
           1       0.52      0.83      0.64       294

    accuracy                           0.52       575
   macro avg       0.52      0.51      0.46       575
weighted avg       0.52      0.52      0.46       575

[[ 54 227]
 [ 49 245]]


RANDOM FOREST

In [566]:
from sklearn.ensemble import RandomForestClassifier  # O RandomForestRegressor si es un problema de regresión
from sklearn.metrics import accuracy_score, classification_report  # Importa otras métricas según tu caso


In [567]:
# Importar las librerías necesarias
from sklearn.ensemble import RandomForestClassifier

#AllVar_df =AllVar_df.loc['2019-04-03':].copy ()

fecha_inicio = '2012-01-01'
AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]

# Definir las variables independientes X y la variable dependiente Y
X_RF = AllVar_df_Shortened.drop(columns=['Next_Day_Returns','TARGET'])
Y_RF = AllVar_df_Shortened['TARGET']


In [568]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_RF, X_test_RF, Y_train_RF, Y_test_RF = train_test_split(X_RF, Y_RF, test_size=0.1, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train_RF)
X_train_RF_scaled = scaler.transform(X_train_RF)
X_test_RF_scaled = scaler.transform(X_test_RF)

In [474]:
# Crear una instancia del modelo Random Forest
# Ajusta los hiperparámetros como n_estimators (número de árboles) y max_depth (profundidad máxima de cada árbol) según necesites
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con el conjunto de entrenamiento escalado
rf_model.fit(X_train_RF_scaled, Y_train_RF)


RandomForestClassifier(random_state=42)

In [569]:
# Predecir las etiquetas para el conjunto de prueba
Y_pred_RF = rf_model.predict(X_test_RF_scaled)

# Calcular la precisión del modelo
accuracy = accuracy_score(Y_test_RF, Y_pred_RF)
print(f"Accuracy of the Random Forest model: {accuracy:.2f}")

# Otras métricas de evaluación
print(classification_report(Y_test_RF, Y_pred_RF))


Accuracy of the Random Forest model: 0.47
              precision    recall  f1-score   support

           0       0.46      0.97      0.62       130
           1       0.67      0.05      0.09       158

    accuracy                           0.47       288
   macro avg       0.56      0.51      0.36       288
weighted avg       0.57      0.47      0.33       288



In [570]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_RF_SV, X_test_RF_SV, Y_train_RF_SV, Y_test_RFSV = train_test_split(X_RF, Y_RF, test_size=0.1, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train_RF_SV)
X_train_RF_SV_scaled = scaler.transform(X_train_RF_SV)
X_test_RF_SV_scaled = scaler.transform(X_test_RF_SV)

In [571]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np

# Suponiendo que X_RF y Y_RF ya están definidos y son tus datos originales
X_train_RF_SV, X_test_RF_SV, Y_train_RF_SV, Y_test_RF_SV = train_test_split(X_RF, Y_RF, test_size=0.1, random_state=42)

# Escalar los datos
scaler = StandardScaler()
scaler.fit(X_train_RF_SV)  # Asegurarte de ajustar el scaler a X_train_RF_SV
X_train_RF_SV_scaled = scaler.transform(X_train_RF_SV)
X_test_RF_SV_scaled = scaler.transform(X_test_RF_SV)

# Verificar si X_train_RF_SV es un DataFrame y extraer los nombres de las columnas
if isinstance(X_train_RF_SV, pd.DataFrame):
    feature_names = X_train_RF_SV.columns.tolist()
else:
    raise ValueError("X_train_RF_SV debe ser un DataFrame para extraer nombres de columnas.")

# Convertir arrays de NumPy a DataFrames para facilitar la manipulación
X_train_RF_SV_scaled_df = pd.DataFrame(X_train_RF_SV_scaled, columns=feature_names)
X_test_RF_SV_scaled_df = pd.DataFrame(X_test_RF_SV_scaled, columns=feature_names)

# Crear el modelo de Random Forest
model = RandomForestClassifier(n_estimators=50, random_state=42)

# Crear el objeto RFE y clasificar las características más importantes
rfe = RFE(estimator=model, n_features_to_select=10)
rfe.fit(X_train_RF_SV_scaled_df, Y_train_RF_SV)

# Crear un DataFrame para mostrar los resultados usando los nombres de las características
results_df = pd.DataFrame({
    'Feature': feature_names,
    'Selected': rfe.support_,
    'Ranking': rfe.ranking_
})

# Filtrar para mostrar sólo las características seleccionadas
selected_features_df = results_df[results_df['Selected'] == True]

# Imprimir las características seleccionadas y su ranking
print("Selected features and their rankings:")
print(selected_features_df)


Selected features and their rankings:
                 Feature  Selected  Ranking
11               DEXUSEU      True        1
14               DEXMXUS      True        1
31  DEXMXUS_daily_change      True        1
32   VIXCLS_daily_change      True        1
38                Volume      True        1
43              Momentum      True        1
46                RSI_20      True        1
47                RSI_10      True        1
52         Lag 1_Returns      True        1
53         Lag 2_Returns      True        1


In [572]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def optimize_rf_with_feature_selection(X_train_opt, y_train_opt):
    if isinstance(X_train_opt, np.ndarray):
        X_train_opt = pd.DataFrame(X_train_opt, columns=['feature_' + str(i) for i in range(X_train_opt.shape[1])])

    # Definir los parámetros para la búsqueda
    params = {
        'n_estimators': (10, 1000),
        'max_depth': (1, 100),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 20),
        'max_features': ('sqrt', 'log2', 0.1, 0.5, 0.9),  # Ajuste de parámetros válidos
    }
    
    clf = RandomForestClassifier(random_state=42)
    selector = SelectFromModel(clf)
    opt = BayesSearchCV(clf, params, n_iter=50, cv=3, random_state=42)
    opt.fit(X_train_opt, y_train_opt)

    best_estimator = opt.best_estimator_
    selector.estimator = best_estimator
    selector.fit(X_train_opt, y_train_opt)
    X_selected = selector.transform(X_train_opt)
    selected_features = X_train_opt.columns[selector.get_support()]

    return best_estimator, selected_features, X_selected

# Preparación y escalado de datos
X_train_RF_opt, X_test_RF_opt, y_train_RF_opt, y_test_RF_opt = train_test_split(X_RF, Y_RF, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_RF_opt_scaled = scaler.fit_transform(X_train_RF_opt)
X_test_RF_opt_scaled = scaler.transform(X_test_RF_opt)

# Optimización y selección de características
best_estimator, selected_features, X_train_RF_opt_scaled_selected = optimize_rf_with_feature_selection(X_train_RF_opt_scaled, y_train_RF_opt)

# Entrenamiento del modelo final
X_train_RF_opt_scaled_selected_df = pd.DataFrame(X_train_RF_opt_scaled_selected, columns=selected_features)
best_estimator.fit(X_train_RF_opt_scaled_selected_df, y_train_RF_opt)

# Preparación del conjunto de prueba
X_test_RF_opt_scaled_selected = pd.DataFrame(X_test_RF_opt_scaled, columns=[f'feature_{i}' for i in range(X_test_RF_opt_scaled.shape[1])])[selected_features]

# Evaluación del modelo
y_pred = best_estimator.predict(X_test_RF_opt_scaled_selected)
accuracy = accuracy_score(y_test_RF_opt, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test_RF_opt, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test_RF_opt, y_pred))


Accuracy: 0.55
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.12      0.19       257
           1       0.56      0.89      0.69       318

    accuracy                           0.55       575
   macro avg       0.52      0.51      0.44       575
weighted avg       0.52      0.55      0.47       575

Confusion Matrix:
[[ 31 226]
 [ 34 284]]


LSTM

In [557]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical

In [558]:
# Definir las variables independientes X y la variable dependiente Y
#X_LSTM = AllVar_df.drop(columns=['Next_Day_Returns','TARGET', 'DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS_daily_change','GVZCLS_daily_change'])
fecha_inicio = '2012-01-01'
AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]
X_LSTM = AllVar_df_Shortened[['DEXCAUS_daily_change', 'DEXMXUS_daily_change', 'Close', 'Volume', 'RSI_20', 'RSI_10', 'Williams_%R', 'Daily_Returns', 'Lag 1_Returns', 'Lag 2_Returns','VIXCLS','GVZCLS','MACD','Signal_Line','Stochastic_%K','Stochastic_%D','DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE']]
Y_LSTM = AllVar_df_Shortened['TARGET']


In [559]:
X_LSTM.head()

,DEXCAUS_daily_change,DEXMXUS_daily_change,Close,Volume,RSI_20,RSI_10,Williams_%R,Daily_Returns,Lag 1_Returns,Lag 2_Returns,...,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,BAMLH0A2HYB,BAMLH0A3HYC,BAMLHE00EHYIOAS,T10Y2Y,T10Y3M,T5YIE
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-04,0.0045,0.0232,1277.300049,3592580000,45.629419,18.545107,-9.277496,0.000188,0.015474,-0.004291,...,1.89,5.30,3.30,7.00,6.98,13.06,10.01,1.75,1.98,1.71
2012-01-05,0.0063,0.0534,1281.060059,4315950000,45.166664,25.809245,-6.459694,0.002944,0.000188,0.015474,...,1.89,5.29,3.27,7.00,6.98,13.10,10.02,1.75,2.00,1.74
2012-01-06,0.0034,-0.0337,1277.810059,3656830000,46.416378,29.466339,-16.650202,-0.002537,0.002944,0.000188,...,1.87,5.25,3.27,7.00,6.97,13.08,9.95,1.73,1.96,1.71
2012-01-09,0.0041,0.0179,1280.699951,3371600000,38.997313,32.346000,-10.895070,0.002262,-0.002537,0.002944,...,1.88,5.26,3.28,6.99,6.96,13.05,9.97,1.72,1.97,1.73
2012-01-10,-0.0111,-0.1133,1292.079956,4221960000,40.831147,32.324606,-9.159368,0.008886,0.002262,-0.002537,...,1.86,5.25,3.25,6.92,6.90,12.90,9.90,1.76,1.98,1.70


In [560]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Supongamos que tienes un DataFrame llamado X_LSTM con características y otro llamado Y_LSTM con etiquetas

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train_LSTM, X_test_LSTM, Y_train_LSTM, Y_test_LSTM = train_test_split(X_LSTM, Y_LSTM, test_size=0.2, random_state=42)

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar el escalador a los datos de entrenamiento y transformarlos
X_train_scaled_LSTM = scaler.fit_transform(X_train_LSTM)
X_test_scaled_LSTM = scaler.transform(X_test_LSTM)

# Cambiar la forma de X_train y X_test para LSTM: [muestras, pasos de tiempo, características]
X_train_LSTM_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_LSTM_reshaped= X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Ahora tienes los conjuntos de entrenamiento y prueba listos para usar en tu red LSTM


In [561]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    # Primera capa LSTM con return_sequences=True para permitir más capas LSTM
    LSTM(50, activation='relu', input_shape=(X_train_LSTM_reshaped.shape[1], X_train_LSTM_reshaped.shape[2]), return_sequences=True),
    Dropout(0.2),  # Dropout para evitar el sobreajuste

    # Segunda capa LSTM. Si esta es tu última capa LSTM, establece return_sequences=False
    LSTM(50, activation='relu', return_sequences=False),  # Cambio clave para la última capa LSTM
    Dropout(0.2),

    # Capa densa final para clasificación binaria
    Dense(1, activation='sigmoid')  # Función de activación sigmoid para salida binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [562]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    # Primera capa LSTM con return_sequences=True para permitir más capas LSTM
    LSTM(50, activation='relu', input_shape=(X_train_LSTM_reshaped.shape[1], X_train_LSTM_reshaped.shape[2]), return_sequences=True),
    Dropout(0.2),  # Dropout para evitar el sobreajuste

    # Segunda capa LSTM
    LSTM(50, activation='relu', return_sequences=True),  # Cambio clave para permitir más capas LSTM
    Dropout(0.2),

    # Tercera capa LSTM (última capa)
    LSTM(50, activation='relu', return_sequences=False),  # Cambio clave para la última capa LSTM
    Dropout(0.2),

    # Capa densa final para clasificación binaria
    Dense(1, activation='sigmoid')  # Función de activación sigmoid para salida binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [563]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential([
        LSTM(
            hp.Int('units_1', min_value=30, max_value=100, step=10),
            activation='relu',
            input_shape=(X_train_LSTM_reshaped.shape[1], X_train_LSTM_reshaped.shape[2]),
            return_sequences=True
        ),
        Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)),

        LSTM(
            hp.Int('units_2', min_value=30, max_value=100, step=10),
            activation='relu',
            return_sequences=True
        ),
        Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)),

        LSTM(
            hp.Int('units_3', min_value=30, max_value=100, step=10),
            activation='relu',
            return_sequences=False
        ),
        Dropout(hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Crear el objeto tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='lstm_optimization'
)

# Inicia la búsqueda de hiperparámetros
tuner.search(X_train_LSTM_reshaped, Y_train_LSTM, epochs=10, validation_split=0.2)

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Reloading Tuner from my_dir\lstm_optimization\tuner0.json


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 80)          │        37,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 80)          │        51,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 40)             │        19,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,681 (424.54 KB)

 Trainable params: 108,681 (424.54 KB)

 Non-trainable params: 0 (0.00 B)

In [564]:
# Entrenar el mejor modelo con los datos completos de entrenamiento
history = best_model.fit(
    X_train_LSTM_reshaped,
    Y_train_LSTM,
    epochs=50,  # Puedes ajustar el número de épocas según sea necesario
    validation_split=0.2,  # O usar X_test_LSTM_reshaped, Y_test_LSTM para validación
    batch_size=32  # Ajusta el tamaño de lote según las necesidades
)


Epoch 1/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5599 - loss: 0.6778 - val_accuracy: 0.5500 - val_loss: 1.2987
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5809 - loss: 0.6697 - val_accuracy: 0.5391 - val_loss: 1.3295
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5701 - loss: 0.6675 - val_accuracy: 0.5239 - val_loss: 1.1954
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6161 - loss: 0.6649 - val_accuracy: 0.5326 - val_loss: 1.2612
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5763 - loss: 0.6637 - val_accuracy: 0.5391 - val_loss: 1.3172
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5895 - loss: 0.6518 - val_accuracy: 0.5130 - val_loss: 1.4836
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6158 - loss: 0.6427 - val_accuracy: 0.4783 - val_loss: 1.3216
Epoch 8/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6181 - loss: 0.6470 - val_accuracy: 0.4674 - val_loss:

In [565]:
# Evaluar el modelo con los datos de prueba
loss, accuracy = best_model.evaluate(X_test_LSTM_reshaped, Y_test_LSTM)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7188 - loss: 9.5883

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5794 - loss: 12.8316
Test Loss: 11.116490364074707
Test Accuracy: 0.5547826290130615
